In [1]:
from IPython.display import display

In [10]:
from dsmlibrary.datanode import DataNode
from tqdm.auto import tqdm
import requests
import pandas as pd
import dask.dataframe as dd

In [3]:
datanode = DataNode()

Init DataNode sucessful!


In [4]:
landing_dir_id = 16
staging_dir_id = 17

### Get Data

#### Board Data

In [27]:
board_data_fID = datanode.get_file_id(directory_id=landing_dir_id ,name="set-board.parquet")
ddf_board_data = datanode.read_ddf(file_id=board_data_fID)
print(ddf_board_data.shape)
ddf_board_data.head()

(Delayed('int-2c5ce4d3-915e-40ac-aac7-530b79c4870f'), 3)


,name,positions,company
0,นาย ครรชิต สิงห์สุวรรณ์,"ประธานกรรมการ,กรรมการอิสระ,กรรมการตรวจสอบ",2s
0,นาย โนบูโยชิ อิดะ,"ประธานกรรมการบริษัท,กรรมการผู้จัดการ",cpr
0,นาย สุเทพ วงศ์วรเศรษฐ,ประธานกรรมการ,mk
0,นาย อภิชาต ลี้อิสสระนุกูล,ประธานกรรมการ,stanly
0,นาย วิสูตร เอี้ยวศิวิกูล,ประธานกรรมการ,ml


#### Shareholder Data

In [28]:
shareholder_fID = datanode.get_file_id(directory_id=landing_dir_id ,name="set-shareholder.parquet")
ddf_shareholder = datanode.read_ddf(file_id=shareholder_fID)
print(ddf_shareholder.shape)
ddf_shareholder.head()

(Delayed('int-0aeb7a87-4743-4164-9372-2a5231b2fa91'), 7)


,sequence,name,nationality,numberOfShare,percentOfShare,isThaiNVDR,company
0,1,นาย สมบัติ ลีสวัสดิ์ตระกูล,,95234050,19.05,False,2s
0,1,UOB KAY HIAN PRIVATE LIMITED,,205467426,12.91,False,thl
0,1,บริษัท เมืองไทยประกันชีวิต จำกัด (มหาชน),,59649200,10.00,False,pl
0,1,นาย จิรวุฒิ คุวานันท์,,325311216,45.45,False,b52
0,1,UBS AG SINGAPORE BRANCH,,1226812593,32.89,False,ea


### Merge

In [29]:
ddf_shareholder_board = ddf_board_data.merge(ddf_shareholder, on='name')
print(ddf_shareholder_board.shape)

(Delayed('int-e5d6c78a-4026-43a8-8988-7190ee4be66b'), 9)


In [30]:
ddf_shareholder_board['positions'] = ddf_shareholder_board['positions'].str.split(',')
ddf_shareholder_board = ddf_shareholder_board.explode('positions')
ddf_shareholder_board = ddf_shareholder_board.rename(columns={'company_x': 'company'})
ddf_shareholder_board = ddf_shareholder_board.drop(columns=['company_y', 'isThaiNVDR'])
ddf_shareholder_board

,name,positions,company,sequence,nationality,numberOfShare,percentOfShare
npartitions=1,,,,,,,
,object,object,object,int64,object,int64,float64
,...,...,...,...,...,...,...


In [31]:
ddf_shareholder_board.compute()

,name,positions,company,sequence,nationality,numberOfShare,percentOfShare
0,นาย อภิชาต ลี้อิสสระนุกูล,ประธานกรรมการ,stanly,2,,5468771,7.14
1,นาย อภิชาต ลี้อิสสระนุกูล,ประธานกรรมการ,stanly,7,,4697700,2.35
2,นาย อภิชาต ลี้อิสสระนุกูล,กรรมการ,leo,2,,5468771,7.14
3,นาย อภิชาต ลี้อิสสระนุกูล,กรรมการ,leo,7,,4697700,2.35
4,นาย อภิชาต ลี้อิสสระนุกูล,กรรมการ,irc,2,,5468771,7.14
...,...,...,...,...,...,...,...
2007,นาย ชวลิต เศรษฐเมธีกุล,ประธานกรรมการตรวจสอบ,bdms,1,,80000000,6.11
2008,นาย ธีธัช ธีระกุล,กรรมการอิสระ,polar,9,,2710000,0.45
2009,น.ส. ภัทรวรรณ มานัสสถิตย์,กรรมการอิสระ,tvo,3,,63706339,10.00
2009,น.ส. ภัทรวรรณ มานัสสถิตย์,กรรมการตรวจสอบ,tvo,3,,63706339,10.00


##### Write Data Node (Parquet)

In [35]:
datanode.write(
    df=ddf_shareholder_board, 
    directory=staging_dir_id, 
    name="set-shareholder-board", 
    profiling=True, 
    lineage=[board_data_fID, shareholder_fID]
)

{'sucess': True,
 'file_id': 65,
 'path': 'SothanaV/set-scrapping/staging/set-shareholder-board.parquet',
 'lineage': True}

##### Write CSV

In [33]:
ddf_shareholder_board.compute().to_csv('dsm.tmp/shareholder_board.csv', index=False)

In [34]:
datanode.upload_file(directory_id=staging_dir_id, file_path='dsm.tmp/shareholder_board.csv', description="รายชื่อบุคคลที่เป็นทั้งกรรมการบริษัทและเป็นผู้ถือหุ้นติด 10 อันดับแรกในบริษัทตัวเองจาก SET")

  0%|          | 0.00/358k [00:00<?, ?B/s]

{'id': 64,
 'msg': 'file shareholder_board.csv create sucessful',
 'path': 'SothanaV/set-scrapping/staging/shareholder_board.csv',
 'uploaded': True}